<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [15]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import math
import pandas as pd
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
from skimage.measure import shannon_entropy
import pandas as pd
import os
from skimage.feature import greycomatrix as gray
from skimage.feature import greycoprops as props
import cv2 as cv

import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import pydicom
from sklearn.metrics import roc_auc_score
from copy import deepcopy
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
#For this function, pass the original image without normalization
def calc_entropy(img):
    image_size = img.size
    values_count = []
    for i in range(np.min(img), np.max(img) + 1):
        occurrences = np.count_nonzero(img == i)
        values_count.append(occurrences)
    image_size -= values_count[0]
    values_count = [i/image_size for i in values_count]
    s = 0
    for i in range(1, len(values_count)):
        if values_count[i] == 0:
            pass
        else:
            s = -1*values_count[i]*math.log2(values_count[i]) + s
    return s

In [3]:
def graycoprops(glcm):
    levels = glcm.shape[0]
    I, J = np.ogrid[0:levels, 0:levels]
    
    contrast_weights = (I - J) ** 2
    
    dissimilarity_weights = np.abs(I - J)
    
    homogeneity_weights = 1. / (1. + (I - J) ** 2)
    
    diff_i = I - np.apply_over_axes(np.sum, (I*glcm), axes = (0,1))[0, 0]
    diff_j = J - np.apply_over_axes(np.sum, (J*glcm), axes = (0,1))[0, 0]
    std_i = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_i) ** 2),
                                           axes=(0, 1))[0, 0])
    std_j = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_j) ** 2),
                                           axes=(0, 1))[0, 0])
    cov = np.apply_over_axes(np.sum, (glcm * (diff_i * diff_j)),
                                 axes=(0, 1))[0, 0]
    
    correlation = cov / (std_i * std_j)
    
    #Angular Second Moment
    asm = np.apply_over_axes(np.sum, (glcm ** 2), axes = (0, 1))[0, 0]
    
    #Energy
    energy = np.sqrt(asm)
    
    #Contrast
    contrast = np.apply_over_axes(np.sum, (glcm*contrast_weights), 
                                  axes = (0,1))[0][0]
    
    #Dissimilarity
    dissimilarity = np.apply_over_axes(np.sum, (glcm*dissimilarity_weights), 
                                  axes = (0,1))[0][0]
    
    #Homogeneity
    homogeneity = np.apply_over_axes(np.sum, (glcm*homogeneity_weights), 
                                  axes = (0,1))[0][0]
    
    return asm, energy, contrast, dissimilarity, homogeneity, correlation

In [4]:
#Creation of GLCM matrix for excluding the background

#Pass image and distance to be calculated
def graycomatrix_thresh(img, distance):
    max_val = np.amax(img)
    min_val = np.amin(img)
    img = img + min_val*-1
    rows = img.shape[0]
    cols = img.shape[1]
    max_val = np.amax(img)
    min_val = np.amin(img)
    size_glcm = 0
    if min_val == 0:
        size_glcm = max_val + 1
    elif min_val < 0:
        size_glcm = max_val + 1 + min_val*-1
    else:
        size_glcm = max_val
    glcm = np.zeros((size_glcm, size_glcm))
    f = img.flatten()
    l = len(f)
    c = -1
    while(c < l-1):
        c += 1
        if f[c] == min_val:
            pass
        else:
            if c%cols >= cols-distance:
                pass
            else:
                glcm[f[c]][f[c + distance]]+= 1
    glcm = glcm/(np.apply_over_axes(np.sum, glcm, axes = (0, 1)))
    return glcm

In [5]:
def calculate_textures(pp_img, distance, patient_no, status, count):
    glcm = graycomatrix_thresh(pp_img, distance)
    cols = ['Patient', 'Slice no', 'Distance', 'ASM', 'Energy', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Correlation', 'Entropy', 'Status']
    texture_values = list(graycoprops(glcm))
    texture_values.insert(0, distance)
    texture_values.insert(0, count)
    texture_values.insert(0, patient_no)
    texture_values.append(calc_entropy(pp_img))
    texture_values.append(status)
    mydict = {k:v for (k, v) in zip(cols, texture_values)}
    return mydict

In [6]:
def preprocessing(path):
    DICOM = pydicom.dcmread(path)
    img = DICOM.pixel_array
    size = img.shape[0]
    
    offset=1024
    gray1=img.astype(np.int16)-offset
    gray2=np.zeros((size,size),dtype=np.int16)

    for i in range(size):
        for j in range(size):
            if gray1[i][j]>=-200 and gray1[i][j]<=200:
                gray2[i][j]=255
            else:
                pass

    gray_ex=np.zeros([size,size],dtype=np.uint16)
    for i in range(size):
        for j in range(size):
            if gray2[i][j]==255:
                gray_ex[i][j]=img[i][j]
            else:
                pass    

    gray_ex= cv.medianBlur(gray_ex,5)
    
    return gray_ex

In [7]:
def dataset_tool(path):
    subdirectory2 = []
    for i in os.listdir(basepath):
        if os.path.isdir(os.path.join(basepath, i)):
            subdirectory1 = os.path.join(basepath, i)
            for j in (os.listdir(subdirectory1)):
                if os.path.isdir(os.path.join(subdirectory1, j)):
                    subdirectory2.append(os.path.join(subdirectory1, j))
    directories = sorted(subdirectory2)
    counter = 0
    d = {}
    for i, j in zip(range(0, len(directories) - 1, 2), range(1, len(directories), 2)):
        false = directories[i]
        true = directories[j]
        true_directories = []
        false_directories = []
        for entry in os.listdir(true):
            if os.path.isfile(os.path.join(true, entry)):
                if entry != '.DS_Store':
                    true_directories.append(os.path.join(true, entry))

        for entry in os.listdir(false):
            if os.path.isfile(os.path.join(false, entry)):
                if entry != '.DS_Store':
                    false_directories.append(os.path.join(false, entry))

        counter += 1
        d[counter] = {}
        d[counter]['true'] = true_directories
        d[counter]['false'] = false_directories
    no_patients = d.keys()
    cols = ['Patient', 'Slice no', 'Distance', 'ASM', 'Energy', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Correlation', 'Entropy', 'Status']
    count = 0
    df = pd.DataFrame(columns = cols)
    for i in no_patients:
        temp1, temp2 = d[i].keys()
        for j, k in zip(d[i][temp1], d[i][temp2]):
            count += 1
            pp_true = preprocessing(j) 
            pp_false = preprocessing(k)

            df = df.append(calculate_textures(pp_true, 1, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 1, i, 0, count), ignore_index = True)
            
            df = df.append(calculate_textures(pp_true, 2, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 2, i, 0, count), ignore_index = True)
            
            df = df.append(calculate_textures(pp_true, 3, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 3, i, 0, count), ignore_index = True)
        display(df)
    return df

In [8]:
basepath = '/Users/rt/Desktop/files'
df = dataset_tool(basepath)
df.to_csv('/Users/rt/Desktop/dataset.csv')

,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
67,1.0,12.0,1.0,0.000645,0.025390,28135.164209,36.976259,0.318996,0.473686,7.692723,0.0
68,1.0,12.0,2.0,0.000192,0.013839,43650.518726,63.617429,0.148242,0.445561,7.711466,1.0
69,1.0,12.0,2.0,0.000335,0.018305,57790.826529,71.548022,0.181748,0.377404,7.692723,0.0
70,1.0,12.0,3.0,0.000139,0.011791,65083.522198,89.125902,0.095948,0.383519,7.711466,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
259,2.0,44.0,1.0,0.000353,0.018784,23889.052500,36.063644,0.252328,0.504393,7.775325,0.0
260,2.0,44.0,2.0,0.000219,0.014791,70161.032442,89.146069,0.154662,0.402322,7.923590,1.0
261,2.0,44.0,2.0,0.000187,0.013658,48743.263158,68.135179,0.140279,0.416460,7.775325,0.0
262,2.0,44.0,3.0,0.000185,0.013606,102997.592387,125.742713,0.099141,0.354705,7.923590,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
307,3.0,52.0,1.0,0.000260,0.016134,19720.097562,37.309411,0.243468,0.541586,7.964035,0.0
308,3.0,52.0,2.0,0.000126,0.011209,34012.978851,57.791140,0.140138,0.444649,7.888078,1.0
309,3.0,52.0,2.0,0.000124,0.011141,38876.031849,64.706981,0.139190,0.432126,7.964035,0.0
310,3.0,52.0,3.0,0.000090,0.009489,50337.645265,79.028162,0.091102,0.374777,7.888078,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
655,4.0,110.0,1.0,0.000303,0.017393,23755.298496,38.287122,0.225743,0.536442,7.787669,0.0
656,4.0,110.0,2.0,0.000165,0.012861,43666.346654,64.757340,0.138908,0.470692,7.736517,1.0
657,4.0,110.0,2.0,0.000184,0.013549,47916.351054,68.194471,0.142649,0.461355,7.787669,0.0
658,4.0,110.0,3.0,0.000125,0.011167,65308.666483,90.805857,0.091622,0.415251,7.736517,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
685,5.0,115.0,1.0,0.000534,0.023109,19277.412781,27.981955,0.324344,0.549210,7.851881,0.0
686,5.0,115.0,2.0,0.000202,0.014204,42428.717880,58.970904,0.160020,0.452632,7.940008,1.0
687,5.0,115.0,2.0,0.000272,0.016500,38918.600680,53.178887,0.188896,0.430190,7.851881,0.0
688,5.0,115.0,3.0,0.000156,0.012491,62128.147017,83.987498,0.105589,0.395278,7.940008,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
775,6.0,130.0,1.0,0.000357,0.018902,18361.812687,29.529753,0.275022,0.529164,7.777848,0.0
776,6.0,130.0,2.0,0.000227,0.015057,41354.846373,53.721044,0.183354,0.377554,7.854156,1.0
777,6.0,130.0,2.0,0.000169,0.013005,37279.498133,54.724179,0.155636,0.412228,7.777848,0.0
778,6.0,130.0,3.0,0.000163,0.012768,61780.498610,77.816534,0.120335,0.317311,7.854156,1.0


,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000338,0.018383,22619.771613,37.636084,0.261268,0.534480,7.821678,1.0
1,1.0,1.0,1.0,0.000374,0.019349,20177.119559,33.165612,0.264537,0.545926,7.780412,0.0
2,1.0,1.0,2.0,0.000157,0.012512,44325.511746,66.719744,0.147493,0.452187,7.821678,1.0
3,1.0,1.0,2.0,0.000181,0.013460,40997.628612,60.512876,0.152135,0.449952,7.780412,0.0
4,1.0,1.0,3.0,0.000110,0.010486,64881.503529,92.002059,0.091236,0.391190,7.821678,1.0
...,...,...,...,...,...,...,...,...,...,...,...
847,7.0,142.0,1.0,0.000364,0.019091,12266.695245,24.687676,0.253209,0.579264,7.681438,0.0
848,7.0,142.0,2.0,0.000188,0.013713,25487.729027,44.233550,0.159562,0.456916,7.741331,1.0
849,7.0,142.0,2.0,0.000184,0.013560,24857.645301,43.017171,0.153731,0.453286,7.681438,0.0
850,7.0,142.0,3.0,0.000126,0.011216,38412.296961,61.690736,0.103327,0.373230,7.741331,1.0
